In [ ]:
import os
import sys
from datetime import datetime

# Diretório base
BASE_DIR = os.path.dirname(os.path.abspath('.'))
print(f"Diretório base: {os.getcwd()}")

## 1. Verificar diretório de logs

In [ ]:
log_dir = os.path.join(os.getcwd(), "logs")
print(f"Diretório de logs: {log_dir}")
print(f"Existe: {os.path.exists(log_dir)}")

if os.path.exists(log_dir):
    files = os.listdir(log_dir)
    print(f"\nArquivos no diretório ({len(files)}):")
    for f in files:
        full_path = os.path.join(log_dir, f)
        size = os.path.getsize(full_path)
        print(f"  - {f} ({size} bytes)")
else:
    print("Diretório não existe ainda")

## 2. Testar criação de arquivo de log

In [ ]:
# Simular criação de log como o receiver faz
try:
    os.makedirs(log_dir, exist_ok=True)
    print(f"✓ Diretório criado/verificado: {log_dir}")
    
    filename = f"test_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    filepath = os.path.join(log_dir, filename)
    
    log_file = open(filepath, 'w', encoding='utf-8')
    log_file.write("timestamp,cpu_usage,cpu_temp,gpu_load,gpu_temp,ram_percent,ping_ms\n")
    print(f"✓ Arquivo criado: {filepath}")
    
    # Escrever linha de teste
    test_line = f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')},50.0,65.0,30.0,55.0,70.0,15.0\n"
    log_file.write(test_line)
    log_file.flush()
    print(f"✓ Linha de teste escrita")
    
    log_file.close()
    print(f"✓ Arquivo fechado")
    
    # Verificar conteúdo
    with open(filepath, 'r') as f:
        content = f.read()
    print(f"\nConteúdo do arquivo:")
    print(content)
    
except Exception as e:
    print(f"✗ ERRO: {e}")
    import traceback
    traceback.print_exc()

## 3. Verificar permissões de escrita

In [ ]:
import stat

# Verificar permissões do diretório
if os.path.exists(log_dir):
    dir_stat = os.stat(log_dir)
    print(f"Permissões do diretório logs:")
    print(f"  Mode: {oct(dir_stat.st_mode)}")
    print(f"  Writable: {os.access(log_dir, os.W_OK)}")
    print(f"  Readable: {os.access(log_dir, os.R_OK)}")
else:
    print("Diretório não existe")

# Verificar permissões do diretório pai
parent_dir = os.getcwd()
print(f"\nPermissões do diretório pai ({parent_dir}):")
print(f"  Writable: {os.access(parent_dir, os.W_OK)}")

## 4. Simular comportamento do receiver

In [ ]:
# Simular a classe do receiver
class LogTester:
    def __init__(self):
        self.logging_enabled = False
        self.log_file = None
        self.log_dir = os.path.join(os.getcwd(), "logs")
        print(f"LogTester inicializado")
        print(f"  log_dir: {self.log_dir}")
        print(f"  logging_enabled: {self.logging_enabled}")
    
    def toggle_logging(self):
        """Simula _toggle_logging do receiver"""
        self.logging_enabled = not self.logging_enabled
        print(f"\nToggle logging: {self.logging_enabled}")
        
        if self.logging_enabled:
            try:
                os.makedirs(self.log_dir, exist_ok=True)
                filename = f"telemetry_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
                filepath = os.path.join(self.log_dir, filename)
                self.log_file = open(filepath, 'w', encoding='utf-8')
                self.log_file.write("timestamp,cpu_usage,cpu_temp,gpu_load,gpu_temp,ram_percent,ping_ms\n")
                print(f"  ✓ Log iniciado: {filepath}")
            except Exception as e:
                print(f"  ✗ Erro ao criar arquivo: {e}")
                self.logging_enabled = False
        else:
            if self.log_file:
                self.log_file.close()
                self.log_file = None
                print("  ✓ Log encerrado")
    
    def log_data(self, data):
        """Simula _log_to_csv do receiver"""
        if not self.log_file:
            print("  ⚠ log_file é None!")
            return
        
        try:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cpu = data.get("cpu", {})
            gpu = data.get("gpu", {})
            ram = data.get("ram", {})
            net = data.get("network", {})
            
            line = f"{timestamp},{cpu.get('usage', 0)},{cpu.get('temp', 0)},{gpu.get('load', 0)},{gpu.get('temp', 0)},{ram.get('percent', 0)},{net.get('ping_ms', 0)}\n"
            self.log_file.write(line)
            self.log_file.flush()
            print(f"  ✓ Linha escrita: {line.strip()}")
        except Exception as e:
            print(f"  ✗ Erro ao escrever: {e}")

# Testar
tester = LogTester()

In [ ]:
# Ativar logging
tester.toggle_logging()

In [ ]:
# Simular dados recebidos
fake_data = {
    "cpu": {"usage": 45.5, "temp": 62.0},
    "gpu": {"load": 30.0, "temp": 55.0},
    "ram": {"percent": 72.5},
    "network": {"ping_ms": 12.0}
}

print("Escrevendo 5 linhas de teste...")
for i in range(5):
    fake_data["cpu"]["usage"] = 40 + i * 5
    tester.log_data(fake_data)

In [ ]:
# Desativar logging
tester.toggle_logging()

In [ ]:
# Verificar arquivos de log criados
print("Arquivos de log existentes:")
for f in os.listdir(log_dir):
    if f.endswith('.csv'):
        full_path = os.path.join(log_dir, f)
        size = os.path.getsize(full_path)
        print(f"\n📄 {f} ({size} bytes)")
        
        with open(full_path, 'r') as file:
            lines = file.readlines()
            print(f"   Linhas: {len(lines)}")
            for line in lines[:10]:  # Primeiras 10 linhas
                print(f"   {line.strip()}")

## 5. Verificar se está sendo chamado durante update_ui

In [ ]:
# Verificar o código do receiver
with open('receiver_notebook.py', 'r', encoding='utf-8') as f:
    content = f.read()

# Procurar por _log_to_csv
import re

# Encontrar todas as ocorrências
matches = list(re.finditer(r'_log_to_csv|logging_enabled', content))
print(f"Encontradas {len(matches)} ocorrências de '_log_to_csv' ou 'logging_enabled':")

for match in matches:
    # Pegar contexto (linha)
    start = content.rfind('\n', 0, match.start()) + 1
    end = content.find('\n', match.end())
    line = content[start:end]
    line_num = content[:match.start()].count('\n') + 1
    print(f"  Linha {line_num}: {line.strip()}")

## 6. Debug: Verificar fluxo do logging

In [ ]:
# Verificar se a condição 'if self.logging_enabled' está sendo atingida
# no _update_ui

# Encontrar a função _update_ui
update_ui_match = re.search(r'def _update_ui\(self.*?\):(.*?)(?=\n    def |\nclass |$)', content, re.DOTALL)
if update_ui_match:
    update_ui_code = update_ui_match.group(0)
    print("Função _update_ui encontrada:")
    print("="*50)
    # Mostrar apenas as partes relevantes
    for line in update_ui_code.split('\n'):
        if 'log' in line.lower() or 'self.is_connected' in line or 'current_data' in line:
            print(line)
else:
    print("Função _update_ui não encontrada!")

## 7. Conclusões

Possíveis problemas com o log:

1. **O log só é escrito quando há dados recebidos** - Se não está conectado, não escreve
2. **O arquivo de log pode não estar sendo criado** - Verificar permissões
3. **O log_file pode estar None** - A tecla L pode não estar funcionando
4. **O diretório logs pode não existir** - Deveria ser criado automaticamente